In [6]:
import os
import pandas as pd
import numpy as np
import seasonal
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gmean
pd.options.mode.chained_assignment = None
# Seaborn Style
sns.set(style="whitegrid", color_codes=True)

#Fix plots inline for notebook
%matplotlib inline

# Data Location within CC DS
data_location = os.path.join("..","data","external")

#######################################################
# Helpful Functions I may need

def getFiscalYear(dt):
    year = dt.year
    if dt.month>6: year += 1
    return year

def wavg(group, weight_column, value_column):
    d = group[value_column]
    w = group[weight_column]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

#######################################################
# Create Lists of Columns for Subsetting Data for Analysis
# Create Mapping of Column Names for Cleanliness
#

compset_oar_list = [
                   'Date',
                   'Region',
                   'Occ-Trans.',
                   'Occ-Grp.',
                   'Occ-Cont.',
                   'Occ-Total',
                   'ADR-Trans.',
                   'ADR-Grp.',
                   'ADR-Cont.',
                   'ADR-Total',
                   'RevPAR-Trans.',
                   'RevPAR-Grp.',
                   'RevPAR-Cont.',
                    'RevPAR-Total'
                    ]

compset_oar_mapping = {
                   'Occ-Trans.':'OCC_TRANS',
                   'Occ-Grp.':'OCC_GRP',
                   'Occ-Cont.':'OCC_CONT',
                   'Occ-Total':'OCC_TOTAL',
                   'ADR-Trans.':'ADR_TRANS',
                   'ADR-Grp.':'ADR_GRP',
                   'ADR-Cont.':'ADR_CONT',
                   'ADR-Total':'ADR_TOTAL',
                   'RevPAR-Trans.':'RevPAR_TRANS',
                   'RevPAR-Grp.':'RevPAR_GRP',
                   'RevPAR-Cont.':'RevPAR_CONT',
                   'RevPAR-Total':'RevPAR_TOTAL'
                    }

m_cnty_oar_list = [
                   'Year',
                   'Month',
                   'Occ-Trans.',
                   'Occ-Grp.',
                   'Occ-Cont.',
                   'Occ-Total',
                   'ADR-Trans.',
                   'ADR-Grp.',
                   'ADR-Cont.',
                   'ADR-Total',
                   'RevPAR-Trans.',
                   'RevPAR-Grp.',
                   'RevPAR-Cont.',
                   'RevPAR-Total'
                    ]

m_cnty_oar_mapping = {
                   'Occ-Trans.':'OCC_TRANS',
                   'Occ-Grp.':'OCC_GRP',
                   'Occ-Cont.':'OCC_CONT',
                   'Occ-Total':'OCC_TOTAL',
                   'ADR-Trans.':'ADR_TRANS',
                   'ADR-Grp.':'ADR_GRP',
                   'ADR-Cont.':'ADR_CONT',
                   'ADR-Total':'ADR_TOTAL',
                   'RevPAR-Trans.':'RevPAR_TRANS',
                   'RevPAR-Grp.':'RevPAR_GRP',
                   'RevPAR-Cont.':'RevPAR_CONT',
                   'RevPAR-Total':'RevPAR_TOTAL'
                     }

region_mapping = { 
                   'Santa Barbara/Santa Maria, CA':'Santa Barbara County, CA',
                   'San Jose/Santa Cruz, CA':'San Jose-Santa Cruz, CA',
                   'Napa Valley, CA':'Napa County, CA',
                 }

region_exclude = ['Long Beach/Torrance']

small_group =      [
                    'Napa County, CA',
                    'Santa Barbara, CA',
                    'Sonoma County, CA',
                    'South Lake Tahoe, CA',
                    'Laguna Beach, CA',
                    'Newport Beach/Dana Point, CA'
                   ]

large_group =   [
                 'San Diego, CA',
                 'San Francisco/San Mateo, CA',
                 'Palm Springs, CA',
                 'San Jose-Santa Cruz, CA'
                ]

comp_set_amalgam = [
                    'Napa County, CA',
                    'Laguna Beach, CA',
                    'Palm Springs, CA',
                    'San Diego, CA',
                    'Santa Barbara County, CA',
                    'San Francisco/San Mateo, CA',
                    'Sonoma County, CA',
                    'South Lake Tahoe, CA',
                    'San Jose-Santa Cruz, CA',
                    'Newport Beach/Dana Point, CA'
                    ]

# Read in Monterey County XLS
# Documentation for Monterey County XLS goes here...
file_path = os.path.join(data_location, "Monterey-County-City-STR-ALL.xlsx")
m_cnty = pd.read_excel(file_path, sheetname = "County of Monterey 05-16 Seg")

# Read in Compset data
file_path = os.path.join(data_location, "Comp-Set-STR-ALL.xlsx")
compset_hm = pd.read_excel(file_path, sheetname = "Comp-Set OCC-ADR-RevPARv2")
compset_econ = pd.read_excel(file_path, sheetname = "Comp-Set S-D-Rv3")

# Read in Budget data
file_path = os.path.join(data_location, "TID_BUDGET.xlsx")
budget = pd.read_excel(file_path, sheetname = "income")

#Helper Data to expand number of years so they are even for excel viz
file_path = os.path.join(data_location, "Region-Years.xlsx")
region_years = pd.read_excel(file_path, sheetname = "Regions")



#### Demand-Revenue Columns Mappings

S_D_R_mapping = {'Demand-Trans.':'D-TRANS',
                 'Demand-Grp.':'D-GRP',
                 'Demand-Con.':'D-CON',
                 'Demand-Total':'D-TOTAL',
                 'Revenue-Trans.':'R-TRANS',
                 'Revnue-Grp.':'R-GRP',
                 'Revenue-Con.':'R-CON',
                 'Revenue-Total':'R-TOTAL',
                 'Supply-Total':'S-TOTAL'}
S_D_R_cols = ['Year',
              'Region',
              'Month',
              'Date',
              'S-TOTAL',
              'R-TOTAL',
              'D-TOTAL',
              'D-GRP',
              'D-TRANS',
              'D-CON',
              'R-GRP',
              'R-TRANS',
              'R-CON']

In [7]:
###################################################
# Process Competitive Set down, We just want total supply so we can weight our annual averages
compset_econ = compset_econ.rename(columns=S_D_R_mapping)
compset_econ_v1 = compset_econ[S_D_R_cols]
compset_econ_v1['Month'] = compset_econ_v1['Month'].apply(str)
compset_econ_v1['Year'] = compset_econ_v1['Year'].apply(str)
compset_econ_v1['Date'] = compset_econ_v1['Year'] + "-" + compset_econ_v1['Month']
compset_econ_v1['Date'] = pd.to_datetime(compset_econ_v1['Date'], format = '%Y-%m')
compset_econ_v1['Region'] = compset_econ_v1['Region'].str.replace('+','')
compset_econ_v1 = compset_econ_v1.replace({'Region':region_mapping})
#compset_total = compset_econ_v1.groupby('Region')[['Region','Year','Supply-Total','re']].mean()
#compset_total = compset_total.reset_index()

###################################################
# Process monterey_county
m_cnty_v1 = m_cnty[m_cnty_oar_list]
m_cnty_v2 = m_cnty_v1.rename(columns=m_cnty_oar_mapping)
m_cnty_v2['Month'] = m_cnty_v2['Month'].apply(str)
m_cnty_v2['Year'] = m_cnty_v2['Year'].apply(str)
m_cnty_v2['Date'] = m_cnty_v2['Year'] + "-" + m_cnty_v2['Month']
m_cnty_v2['Date'] = pd.to_datetime(m_cnty_v2['Date'], format = "%Y-%b")
m_cnty_v2['Month'] = m_cnty_v2['Date'].apply(lambda x: x.month)
m_cnty_v2['Region'] = 'Monterey County, CA'

###################################################
# Process compset data
compset_hm_v1 = compset_hm = compset_hm[compset_oar_list]
compset_hm_v2 = compset_hm_v1.rename(columns=compset_oar_mapping)

#Date & Time Manipulation
compset_hm_v2['Date'] = pd.to_datetime(compset_hm_v2['Date'])
compset_hm_v2['Month'] = compset_hm_v2['Date'].apply(lambda x: str(x.month))
compset_hm_v2['Year'] = compset_hm_v2['Date'].apply(lambda x: str(x.year))

#String Manipulation
compset_hm_v2['Region'] = compset_hm_v2['Region'].str.replace('+','')


In [8]:
###############################################
# Combine datasets for Hotel Metrics 
compset_hm_v3 = pd.merge(compset_hm_v2, compset_econ_v1, on=['Region','Date','Year','Month'], how = 'right')
tot_set = compset_hm_v3.append(m_cnty_v2)
#tot_set = tot_set.append(m_city_v2) # These datasets differ

# Sort before Calculations (Do I need to do this?)
tot_set = tot_set[(tot_set['Date'] < '2016-07-01') & (tot_set['Date'] > '2009-01-01')]
tot_set = tot_set.sort_values(by=['Date'], ascending = True)
tot_set = tot_set.round(2)
tot_set = tot_set.reset_index()

#This is an IMPORTANT STEP-HERE WE APPLY OUR FISCAL YEAR (Jun-Jul) to the YEAR variable.
# Is this best practices? I don't think so... not sure. need training.
tot_set['Year'] = tot_set['Date'].apply(getFiscalYear)

#String Manipulation, The mappings change over time, so we need to make sure all the names are consistent
tot_set = tot_set.replace({'Region':region_mapping})
tot_set['Month'] = tot_set['Month'].apply(str)

# Drop Duplicates, The City of Monterey is included in the Comp Set as well as the city data, the city data went back further
tot_set = tot_set.drop_duplicates(subset=['Region','Year','Month'], keep='last')

# Regions we want to keep, We only had one period of Long Beach. Exclude it
tot_set = tot_set[~(tot_set['Region'].isin(region_exclude))]

# Let's merge everything together, the comp set data and our data about the county and city
tot_set = pd.merge(tot_set, region_years, on = ['Region','Year'], how = 'right')


In [27]:
######################################
# Lets begin by creating a dataframe whose sole purpose is to create and output 'micro data'

#Micro data can only be calculated post 2011-04-01.
tot_set_v2 = tot_set[(tot_set['Date'] < '2016-07-01') & (tot_set['Date'] > '2011-04-01')]

micro = pd.DataFrame()

micro = tot_set_v2
#Calculate Room nights by multiplying the percent occupancy per segment by total supply of rooms
micro['GRP_RN'] = micro['S-TOTAL']*(micro['OCC_GRP']/100)
micro['TRANS_RN'] = micro['S-TOTAL']*(micro['OCC_TRANS']/100)
micro['TOTAL_RN'] = micro['S-TOTAL']*(micro['OCC_TOTAL']/100)

# Calculate the ADR by dividing the Revenue in each period by Room Nights sold
micro['GRP_ADR'] = micro['R-GRP']/micro['GRP_RN']
micro['TRANS_ADR'] = micro['R-TRANS']/micro['TRANS_RN']
micro['TOTAL_ADR'] = micro['R-TOTAL']/micro['TOTAL_RN']

micro.to_excel("Micro_data.xlsx", sheet_name = 'Raw-Micro')

raw_values= pd.DataFrame()

raw_values['GRP_OCC_ANN'] = micro.groupby(['Region','Year'])['GRP_RN'].sum()/micro.groupby(['Region','Year'])['S-TOTAL'].sum()
raw_values['TRANS_OCC_ANN'] = micro.groupby(['Region','Year'])['TRANS_RN'].sum()/micro.groupby(['Region','Year'])['S-TOTAL'].sum()
raw_values['TOTAL_OCC_ANN'] = micro.groupby(['Region','Year'])['TOTAL_RN'].sum()/micro.groupby(['Region','Year'])['S-TOTAL'].sum()

raw_values['GRP_ADR_ANN'] = micro.groupby(['Region','Year'])['R-GRP'].sum()/micro.groupby(['Region','Year'])['GRP_RN'].sum()

raw_values['GRP_OCC_ANN_PCT'] = raw_values['GRP_OCC_ANN'].pct_change(1)
raw_values['TRANS_OCC_ANN_PCT'] = raw_values['TRANS_OCC_ANN'].pct_change(1)
raw_values['TOTAL_OCC_ANN_PCT'] = raw_values['TOTAL_OCC_ANN'].pct_change(1)

raw_values = raw_values.reset_index()
raw_values.to_excel("Micro_data.xlsx", sheet_name = 'Raw-Annual')


In [26]:
raw_values

,Region,Year,GRP_OCC_ANN,TRANS_OCC_ANN,TOTAL_OCC_ANN,GRP_ADR_ANN,GRP_OCC_ANN_PCT,TRANS_OCC_ANN_PCT,TOTAL_OCC_ANN_PCT
0,"City of Monterey, CA",2012.0,0.231291,0.435478,0.678021,152.747666,NaN,NaN,NaN
1,"City of Monterey, CA",2013.0,0.222537,0.459064,0.693787,157.673217,-0.037846,0.054160,0.023254
2,"City of Monterey, CA",2014.0,0.230847,0.461619,0.703956,159.754816,0.037341,0.005566,0.014656
3,"Laguna Beach, CA",2014.0,0.275697,0.498936,0.800380,NaN,0.194283,0.080839,0.136976
4,"Laguna Beach, CA",2015.0,0.253103,0.495096,0.773628,NaN,-0.081952,-0.007697,-0.033424
5,"Laguna Beach, CA",2016.0,0.210567,0.518527,0.751240,NaN,-0.168055,0.047327,-0.028940
6,"Monterey County, CA",2011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Monterey County, CA",2012.0,0.239464,0.420928,0.667724,161.665165,0.137230,-0.188224,-0.111170
8,"Monterey County, CA",2013.0,0.205910,0.395320,0.609494,157.391335,-0.140122,-0.060838,-0.087208
9,"Monterey County, CA",2014.0,0.227039,0.516181,0.751531,98.932878,0.102615,0.305730,0.233041
